In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV

In [2]:
def set_data(file):
    bikeshare_machine = pd.read_csv(file, 
                        parse_dates=['Start date', 'End date'])
    bikeshare_machine.drop('Unnamed: 0', 1, inplace=True)
    included_cols = ['start_station','end_station','Member Type','time_diff','season','mnth','holiday',
                     'weekday','workingday','weathersit','temp','hum','windspeed','miles',
                     'rush_hour','metro_dist','landmark_dist_start','landmark_dist_end']
    bikeshare_machine = bikeshare_machine[included_cols]
    bikeshare_machine['season'] = bikeshare_machine['season'].astype('category')
    bikeshare_machine['mnth'] = bikeshare_machine['mnth'].astype('category')
    bikeshare_machine['holiday'] = bikeshare_machine['holiday'].astype('category')
    bikeshare_machine['weekday'] = bikeshare_machine['weekday'].astype('category')
    bikeshare_machine['workingday'] = bikeshare_machine['workingday'].astype('category')
    bikeshare_machine['weathersit'] = bikeshare_machine['weathersit'].astype('category')
    bikeshare_machine['Member Type'] = bikeshare_machine['Member Type'].astype('category')
    bikeshare_machine['start_station'] = bikeshare_machine['start_station'].astype('category')
    bikeshare_machine['end_station'] = bikeshare_machine['end_station'].astype('category')
    bikeshare_machine['rush_hour'] = bikeshare_machine['rush_hour'].astype('category')
    col_names = ['start_station', 'end_station','member_type','time_diff','season','month','holiday',
             'weekday','work_day','weather_cat','temperature','humidity','windspeed','miles','rush_hour',
                'metro_dist','landmark_dist_start','landmark_dist_end']
    bikeshare_machine.columns = col_names
    tmin = -8
    tmax = 39
    hum_max = 100
    wind_max = 67
    bikeshare_machine['temp'] = bikeshare_machine['temperature'] * (tmax - tmin) + tmin
    bikeshare_machine['hum'] = bikeshare_machine['humidity'] * 100
    bikeshare_machine['wind'] = bikeshare_machine['windspeed'] * 67
    bikeshare_machine = pd.get_dummies(bikeshare_machine, 
                                 columns=['rush_hour','member_type','holiday','work_day'], drop_first=True)
    bikeshare_machine = pd.get_dummies(bikeshare_machine, 
                                 columns=['start_station','end_station','season','month','weekday','weather_cat'])
    bikeshare_machine = shuffle(bikeshare_machine)
    return bikeshare_machine

In [3]:
file = '/Users/matthewcassi/Documents/Bike-Sharing-Dataset/Bikeshare_Time_Prediction/Casual_RushMetro/landmarks.csv'
bs = set_data(file)

In [4]:
bs.columns

Index(['time_diff', 'temperature', 'humidity', 'windspeed', 'miles',
       'metro_dist', 'landmark_dist_start', 'landmark_dist_end', 'temp', 'hum',
       ...
       'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4',
       'weekday_5', 'weekday_6', 'weather_cat_1', 'weather_cat_2',
       'weather_cat_3'],
      dtype='object', length=303)

In [5]:
bs=bs.rename(columns = {'member_type_Registered':'member_type'})
bs = bs.drop(['temperature','windspeed','humidity'], 1)
bs.head()

,time_diff,miles,metro_dist,landmark_dist_start,landmark_dist_end,temp,hum,wind,rush_hour_1,member_type,...,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weather_cat_1,weather_cat_2,weather_cat_3
532346,15.200,1.709977,0.537488,0.916964,0.257095,24.782500,67.4167,6.999289,0,1,...,0,0,0,0,0,0,1,1,0,0
176562,8.100,0.775454,0.328093,0.306432,0.916964,20.395849,50.7083,18.041961,0,1,...,0,0,1,0,0,0,0,1,0,0
972495,7.783,1.080455,0.508511,1.190711,0.652797,-0.166651,41.9167,16.834286,0,1,...,1,0,0,0,0,0,0,1,0,0
51933,6.317,0.668210,0.485937,0.485473,0.401587,8.143466,68.0000,8.391616,0,1,...,1,0,0,0,0,0,0,1,0,0
247137,8.917,0.771686,0.096430,0.218430,0.235437,23.059151,69.6250,10.333611,0,1,...,0,0,0,1,0,0,0,1,0,0


In [6]:
remove = ['weekday_0', 'weekday_1','weekday_2','weekday_3','weekday_4','weekday_5','weekday_6', 
              'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9',
              'month_10','month_11','month_12','member_type']
X = bs.drop(remove, 1)
y = bs['member_type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((914885, 280), (304962, 280), (914885,), (304962,))

### GridSearch Logistic Regression

In [7]:
n_estimators = 125
class_weight = [{0:.675, 1:.325}, {0:.7, 1:.3}, {0:.68, 1:.32}]
features = ['sqrt','log2',20,30,40]
min_leaf_samples = [50,75,100,125,150]
param_grid = {'class_weight':class_weight, 'max_features': features, 'min_samples_leaf':min_leaf_samples}

rf1 = RandomForestClassifier(n_estimators, n_jobs=-1)
rf_gs1 = RandomizedSearchCV(rf1, param_grid, n_iter=15, cv=3, scoring='roc_auc')
rf_gs1.fit(X_train, y_train)

RandomizedSearchCV(cv=3, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=125, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False),
          fit_params={}, iid=True, n_iter=15, n_jobs=1,
          param_distributions={'class_weight': [{0: 0.675, 1: 0.325}, {0: 0.7, 1: 0.3}, {0: 0.68, 1: 0.32}], 'max_features': ['sqrt', 'log2', 20, 30, 40], 'min_samples_leaf': [50, 75, 100, 125, 150]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score=True, scoring='roc_auc', verbose=0)

In [8]:
pred1 = rf_gs1.predict(X_test)
score1 = rf_gs1.score(X_test, y_test)
train_score1 = rf_gs1.score(X_train, y_train)

In [9]:
score1, train_score1

(0.8618213157491883, 0.88652788893385026)

In [10]:
confusion_matrix(y_test, pred1)

array([[ 28710,  22325],
       [ 16180, 237747]])

In [11]:
print(classification_report(y_test, pred1))

             precision    recall  f1-score   support

          0       0.64      0.56      0.60     51035
          1       0.91      0.94      0.93    253927

avg / total       0.87      0.87      0.87    304962



In [12]:
rf_gs1.best_params_

{'class_weight': {0: 0.7, 1: 0.3}, 'max_features': 40, 'min_samples_leaf': 50}

In [13]:
roc_auc_score(y_test, pred1)

0.74941800443369555